In [45]:
import json
from datetime import timedelta, datetime, date
from typing import Any, Dict, List, Union

import flask
import plaid
from flask import jsonify, make_response
from google.cloud import firestore
from google.cloud.firestore import CollectionReference, DocumentReference
from plaid import ApiException
from plaid.api import plaid_api
from plaid.model.account_balance import AccountBalance
from plaid.model.accounts_get_request import AccountsGetRequest
from plaid.model.accounts_get_response import AccountsGetResponse
from plaid.model.transactions_get_response import TransactionsGetResponse


In [46]:
with open('../../private_keys.json') as f:
    data = json.load(f)
    f.close()

configuration = plaid.Configuration(
    host=plaid.Environment.Development,
    # host=plaid.Environment.Sandbox,
    api_key={
        'clientId': data['client_id'],
        'secret': data['secret_development'],
        # 'secret': data['secret_sandbox'],
    }
)

api_client = plaid.ApiClient(configuration)
plaid_client = plaid_api.PlaidApi(api_client)
firestore_client = firestore.Client.from_service_account_json('../../google-services.json')

In [47]:
def accounts_get(access_token: str) -> Union[List[dict], dict]:
    try:
        request = AccountsGetRequest(access_token)
        response: AccountsGetResponse = plaid_client.accounts_get(request)
        accounts: List[dict] = response['accounts']

        return accounts
    except ApiException as e:
        exceptions: dict = json.loads(e.body)
        return exceptions

In [48]:
def get_linked_account_ids(request: Union[flask.Request, Dict]):
    try:
        if type(request) is flask.Request:
            data = request.data
            data_dict = json.loads(data)
            uid: str = data_dict['uid']
        else:
            uid: str = request['uid']
        
        # Get `accountIds` by accessing user doc on Firestore
        user_doc = firestore_client.collection('users').document(uid)
        user_dict = user_doc.get().to_dict()
        linked_account_ids: List[str] = user_dict.get('accountIds')

        return {'user_doc': user_doc,'account_ids': linked_account_ids, 'uid': uid}
    except Exception as e:
        return {'error': str(e)}

In [49]:
def create_transactions_get_request(user_doc: DocumentReference, account_id: str):
    
    #### Get `access_token` ####
    plaid_keys_collection: CollectionReference = user_doc.collection('plaid_keys')
    access_token_doc = plaid_keys_collection.document(account_id)
    access_token_dict = access_token_doc.get().to_dict()
    access_token = access_token_dict['access_token']

    #### Set `start_date` and `end_date` for request ####
    accounts_collection: CollectionReference = user_doc.collection('accounts')
    account_doc = accounts_collection.document(account_id)
    account_dict = account_doc.get().to_dict()

    if account_dict is None:
        now = datetime.now()
        start_time = now - timedelta(days=60)
        start_date = start_time.date()
    else:
        start_time = account_dict.get('last_synced_time')

        if start_time is None:
            now = datetime.now()
            start_time = now - timedelta(days=60)
            start_date = start_time.date()
        else: 
            start_time = datetime.strptime(
                str(start_time), 
                '%Y-%m-%d %H:%M:%S.%f+00:00'
            )
            start_date = start_time.date()

    end_date = datetime.now().date()

    transactions_get_request = plaid_api.TransactionsGetRequest(
        access_token=access_token,
        start_date=start_date,
        end_date=end_date,
    )

    return transactions_get_request

In [50]:
def update_transactions(uid: str, transactions: List[Dict[str, Any]]):
    try:
        user_doc = firestore_client.collection('users').document(uid)
        transactions_collection = user_doc.collection('transactions')

        for transaction in transactions:
            transaction_id: Union[str, None] = transaction.get('transaction_id')
            transaction_doc: DocumentReference = transactions_collection.document(
                transaction_id
            )
            transaction_snapshot = transaction_doc.get()

            old_date: date = transaction.get('date')
            new_date = datetime(old_date.year,old_date.month, old_date.day)

            authorized_date: Union[date, None] = transaction.get('authorized_date')
            
            if authorized_date is not None:
                new_authorized_date = datetime(authorized_date.year, authorized_date.month, authorized_date.day)
            else:
                new_authorized_date = None

            transaction.update({
                'date': new_date,
                'authorized_date': new_authorized_date,
            })
           
            if transaction_snapshot.exists:
                write_result = transaction_doc.update(transaction)
                update_time = write_result.update_time
            else:
                write_result = transaction_doc.create(transaction)
                update_time = write_result.update_time
                

        return {'status': 200, 'last_update_time': update_time}
    except Exception as e:
        return {'status': 404, 'error_message': str(e)}

In [51]:
def update_accounts(uid: str, accounts: List[dict]) -> Dict[str, Any]:
    try:
        user_doc = firestore_client.collection('users').document(uid)
        accounts_ref: CollectionReference = user_doc.collection('accounts')

        # Update Accounts in `Accounts` collections
        for account in accounts:
            print(f'account {account}')

            # Update Accounts Collections
            account_id: str = account.get('account_id')
            print(f'accountId {account_id}')

            account_doc_ref = accounts_ref.document(account_id)
            account_snapshot = account_doc_ref.get()
            print(f'account_snapshot {account_snapshot}')

            balances: Union[AccountBalance, None] = account.get('balances')
            
            data = {
                'account_id': account.get('account_id'),
                'balances': {
                    'available': balances.get('available'),
                    'current': balances.get('current'),
                    'iso_currency_code': balances.get('iso_currency_code'),
                    'limit': balances.get('limit'),
                    'unofficial_currency_code': balances.get('unofficial_currency_code'),
                },
                'mask': account.get('mask'),
                'name': account.get('name'),
                'official_name': account.get('official_name'),
                'subtype': str(account.get('subtype')),
                'type': str(account.get('type')),
                'verification_status': account.get('verification_status'),
                'last_synced_time': datetime.now()
            }

            if account_snapshot.exists:
                write_result = account_doc_ref.update(data)
                update_time = write_result.update_time
            else:
                write_result = account_doc_ref.set(data)
                update_time = write_result.update_time

            result = {
                'status': 200,
                'last_update_time': update_time,
            }

        return result
    except Exception as e:
        result = {
            'status': 404,
            'error_message': str(e),
        }

        return result

In [52]:
def transactions_refresh(request: flask.Request or dict):

    get_account_ids = get_linked_account_ids(request)

    account_ids = get_account_ids.get('account_ids')
    print(f'Account IDs {account_ids}')

    user_doc = get_account_ids.get('user_doc')
    uid = get_account_ids.get('uid')

    if account_ids is None:
        return make_response(jsonify(error=get_account_ids.get('error')), 404)

    print(f'account_ids {account_ids}')

    fetched_tranactions = 0
    fetched_accounts = 0

    for account_id in account_ids:
        transactions_get_request = create_transactions_get_request(
            user_doc, account_id)
        print(f'request {transactions_get_request}')

        try:
            transactions_get_response: TransactionsGetResponse = plaid_client.transactions_get(
                transactions_get_request
            )
            response_dict = transactions_get_response.to_dict()

            #### Update or Create Transactions Data ####
            transactions = response_dict.get('transactions')
            print(f'Retrieved {len(transactions)} transactions')
            fetched_tranactions += len(transactions)

            accounts = response_dict.get('accounts')
            fetched_accounts += len(accounts)
            print(f'Retrieved {len(accounts)} Accounts')

            #### Update transactions ####
            update_transactions_result = update_transactions(uid, transactions)
            print(f'Update Transactions: {update_transactions_result}')

            #### Update accounts ####
            update_accounts_result = update_accounts(uid, accounts)
            print(f'Update Accounts: {update_accounts_result}')

            transactions_status = update_transactions_result.get('status')
            accounts_status = update_accounts_result.get('status')
            exceptions = None

        except ApiException as e:
            exceptions: dict = json.loads(e.body)
            transactions_status, accounts_status = 404

    result = {
        'transactions_status': transactions_status,
        'accounts_status': accounts_status,
        'fetched_transactions': fetched_tranactions,
        'fetched_accounts': fetched_accounts,
        'exceptions': exceptions,
    }

    return make_response(result, accounts_status)


In [53]:
data = {
    'uid': 's'
}

transactions_refresh(data)

Account IDs ['yB0eX8X9aqf5EZmKJPrktE5xDn4k9LhOoE81V', 'yzdpObDYQqC3x0eLVgM9U540QyLYwJhO3VLvw']
account_ids ['yB0eX8X9aqf5EZmKJPrktE5xDn4k9LhOoE81V', 'yzdpObDYQqC3x0eLVgM9U540QyLYwJhO3VLvw']
request {'access_token': 'access-development-7aea2526-0379-4716-8160-ddc81ea703e3',
 'end_date': datetime.date(2022, 1, 25),
 'start_date': datetime.date(2022, 1, 25)}
Retrieved 0 transactions
Retrieved 1 Accounts
Update Transactions: {'status': 404, 'error_message': "local variable 'update_time' referenced before assignment"}
account {'account_id': 'yzdpObDYQqC3x0eLVgM9U540QyLYwJhO3VLvw', 'balances': {'available': 8774.33, 'current': 8774.33, 'limit': None, 'iso_currency_code': 'USD', 'unofficial_currency_code': None}, 'mask': '5936', 'name': 'CHASE COLLEGE', 'official_name': None, 'type': 'depository', 'subtype': 'checking'}
accountId yzdpObDYQqC3x0eLVgM9U540QyLYwJhO3VLvw
account_snapshot <google.cloud.firestore_v1.base_document.DocumentSnapshot object at 0x7fec43051bb0>
Update Accounts: {'status'

{'transactions_status': 200,
 'accounts_status': 200,
 'fetched_transactions': 1,
 'fetched_accounts': 2,
 'exceptions': None}